In [1]:
import torch
print(torch.cuda.device_count())
#print(torch.cuda.get_device_name(0))
#print(torch.cuda.get_device_properties(0))
#print(torch.cuda.get_device_capability(0))


0


In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import VitsTokenizer, VitsModel, set_seed
from scipy.io.wavfile import write
from IPython.display import Audio
import torchaudio , librosa
import sounddevice as sd
import noisereduce as nr
import soundfile as sf
import numpy as np
import socket
import torch
import scipy
import time



c:\Users\Angry Nerd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
#device="cuda:0" if torch.cuda_is_available() else "cpu"
#model_whisper = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").cuda()
model_whisper = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

In [5]:
#model_vits = VitsModel.from_pretrained("facebook/mms-tts-eng").cuda()
model_vits = VitsModel.from_pretrained("facebook/mms-tts-eng")

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [6]:
print(model_whisper.device)
print(model_vits.device)

cpu
cpu


## Start listening

In [7]:
PORT = 12345
NUM_SAMPLES = 2048
Sampling_Rate = 16000

receiving_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
receiving_socket.bind(("0.0.0.0", PORT))

print("Listening.....");

data = []
data = np.array(data , dtype=np.int8)

stream = sd.OutputStream(channels=1, dtype="int8", samplerate=Sampling_Rate)
stream.start()
start_time=time.time()
while(time.time()-start_time)<30:
    try:
        audio_bytes, server_addr = receiving_socket.recvfrom(NUM_SAMPLES)
        audio_sample = np.frombuffer(audio_bytes, dtype=np.uint8)
        audio_sample = np.array((audio_sample - 128) , dtype=np.int8) 
        audio_sample = nr.reduce_noise(y=audio_sample, sr=Sampling_Rate)
        #audio_sample=process(audio_bytes)
        #print(audio_sample)
        data = np.concatenate((data, audio_sample))
        stream.write(audio_sample)
            
    except:
        receiving_socket.close()
        stream.close()
print("Stream received, Start processing")

forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="translate")

new_data=data.reshape(1,data.shape[0])
scaled_data=new_data/127 # to become in range -1 to 1 and will bocome float64
arr=scaled_data.astype(np.float32)  #as whisper recieve float32 not  float 64
sample_rate=16000
input_features = processor(arr, sampling_rate=sample_rate, return_tensors="pt").input_features 

#input_features = input_features.to(torch.device('cuda'))
# generate token ids
start_time = time.time()
predicted_ids = model_whisper.generate(input_features, forced_decoder_ids=forced_decoder_ids)
translation = processor.batch_decode(predicted_ids, skip_special_tokens=True)

end_time = time.time()
print(translation)

elapsed_time = end_time - start_time
print(f"Process took {elapsed_time:.2f} seconds.")


inputs = tokenizer(text=translation, return_tensors="pt")
#inputs= inputs.to(torch.device('cuda'))

set_seed(555)

with torch.no_grad():
   outputs = model_vits(**inputs)

waveform = outputs.waveform[0].to("cpu")

float_array = waveform.numpy().astype(float)
sd.play(float_array,samplerate=model_vits.config.sampling_rate)
sd.wait()
# sd.play(data,16000)
# sd.wait()

Listening.....
Stream received, Start processing
[' .']
Process took 47.77 seconds.


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)